In [3]:
# Importing Necessary Libraries
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
import os

In [10]:
# Replacing the encoder part with Feature Extraxtor of VGG

# Import the VGG16 model from Keras.
from keras.applications.vgg16 import VGG16

# Create an instance of the VGG16 model.
vggmodel = VGG16()

# Create a new sequential model for feature extraction.
newmodel = Sequential()

# Iterate through the layers of the VGG16 model.
for i, layer in enumerate(vggmodel.layers):
    if i < 19:  # Include only the first 19 layers for feature extraction.
        newmodel.add(layer)

# Display a summary of the new model's architecture.
newmodel.summary()

# Set all layers in the new model to be non-trainable to retain pre-trained weights.
for layer in newmodel.layers:
    layer.trainable = False  # We don't want to train these layers again, so set them to non-trainable.


553476096/553467096 [==============================] - 6s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_______________________________________________________

In [4]:
# VGG16 expects images to have 3 dimensions (height, width, and channels) with a size of 224x224 pixels.

# Define the root directory for the image data.
ROOT_DIR = 'ImageColoring/MyDrive/ImageColoring/'

# Create an ImageDataGenerator for data preprocessing.
# Normalize images by dividing pixel values by 255.
train_datagen = ImageDataGenerator(rescale=1. / 255)

# Create a generator for loading and preprocessing images.
# Flow images from the specified directory, resizing them to 224x224 pixels, and using a batch size of 1000.
# The class_mode is set to None because this is likely for data loading only and not for classification.
train = train_datagen.flow_from_directory(ROOT_DIR, target_size=(224, 224), batch_size=1000, class_mode=None)


Found 7144 images belonging to 2 classes.


---

In [7]:
# Convert from RGB to Lab: LAB image is a grayscale image in L channel and all color info stored in A and B channels
X = []
Y = []

# Iterate through images in the 'train' dataset
for img in train[0]:
    try:
        # Convert the RGB image to LAB color space
        lab = rgb2lab(img)

        # Extract and append the L channel (luminance) to the 'X' list
        X.append(lab[:, :, 0])

        # Extract the A and B channels (color information) and normalize the values
        # A and B values range from -127 to 128, so we divide the values by 128 to restrict them to the range between -1 and 1
        Y.append(lab[:, :, 1:] / 128)
    except:
        # Handle any errors that occur during processing
        print('Error processing an image.')

# Convert the lists to NumPy arrays for use in machine learning models
X = np.array(X)
Y = np.array(Y)

# Add an additional channel to the grayscale images to make their dimensions compatible with the color channels
X = X.reshape(X.shape + (1,))

# Print the shapes of the resulting arrays
print(X.shape)
print(Y.shape)


(1000, 224, 224, 1)
(1000, 224, 224, 2)


---

In [8]:
# We have one channel of L in each layer but, VGG16 is expecting 3 dimension, 
# so we repeat the L channel two times to get 3 dimensions of the same L channel

# Initialize an empty list to store VGG16 features for each L channel
vggfeatures = []

# Iterate through each sample in X
for i, sample in enumerate(X):
    # Convert the single-channel L image to a three-channel grayscale image (RGB)
    sample = gray2rgb(sample)
    
    # Reshape the image to match the expected input shape of the VGG16 model
    sample = sample.reshape((1, 224, 224, 3))
    
    # Make a prediction using the VGG16 model
    prediction = newmodel.predict(sample)
    
    # Reshape the prediction to match the desired shape (7x7x512)
    prediction = prediction.reshape((7, 7, 512))
    
    # Append the VGG16 features to the list
    vggfeatures.append(prediction)

# Convert the list of features to a NumPy array
vggfeatures = np.array(vggfeatures)

# Print the shape of the resulting array
print(vggfeatures.shape)


(1000, 7, 7, 512)


In [9]:
# Create a Sequential model for the decoder
model = Sequential()

# Add a convolutional layer with 256 filters, ReLU activation, and same padding
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', input_shape=(7, 7, 512)))

# Add a convolutional layer with 128 filters, ReLU activation, and same padding
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

# Add an upsampling layer to increase the spatial resolution by a factor of 2
model.add(UpSampling2D((2, 2)))

# Add a convolutional layer with 64 filters, ReLU activation, and same padding
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

# Add an upsampling layer to increase the spatial resolution by a factor of 2
model.add(UpSampling2D((2, 2)))

# Add a convolutional layer with 32 filters, ReLU activation, and same padding
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))

# Add an upsampling layer to increase the spatial resolution by a factor of 2
model.add(UpSampling2D((2, 2)))

# Add a convolutional layer with 16 filters, ReLU activation, and same padding
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))

# Add an upsampling layer to increase the spatial resolution by a factor of 2
model.add(UpSampling2D((2, 2)))

# Add a convolutional layer with 2 filters, tanh activation, and same padding
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

# Add an upsampling layer to increase the spatial resolution by a factor of 2
model.add(UpSampling2D((2, 2))

# Print a summary of the model architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 7, 256)         1179904   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         295040    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 56, 56, 32)       

In [10]:
# Compile the model using the Adam optimizer, mean squared error loss, and accuracy metric
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

# Train the model using VGG features and Y (target) data
model.fit(vggfeatures, Y, verbose=1, epochs=2000, batch_size=16)


Epoch 1/2000
63/63 [==============================] - 3s 16ms/step - loss: 0.0455 - accuracy: 0.6002
Epoch 2/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0104 - accuracy: 0.6775
Epoch 3/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0094 - accuracy: 0.6925
Epoch 4/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0087 - accuracy: 0.7045
Epoch 5/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0082 - accuracy: 0.7088
Epoch 6/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0077 - accuracy: 0.7181
Epoch 7/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0074 - accuracy: 0.7279
Epoch 8/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0068 - accuracy: 0.7375
Epoch 9/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0065 - accuracy: 0.7460
Epoch 10/2000
63/63 [==============================] - 1s 13ms/step - loss: 0.0062 - accura

In [12]:
# Save the model to a specific file path
model.save(ROOT_DIR + 'colorize_autoencoder_2000.model')


INFO:tensorflow:Assets written to: ImageColoring/MyDrive/ImageColoring/colorize_autoencoder_2000.model/assets


In [5]:
ROOT_DIR = 'ImageColoring/MyDrive/ImageColoring/'

In [6]:
# Load the saved model from the specified file path
model = tf.keras.models.load_model(ROOT_DIR + 'colorize_autoencoder_2000.model',
                                   custom_objects=None,
                                   compile=True)


In [8]:
# Get a list of files in the 'testpath' directory
testpath = 'ImageColoring/MyDrive/ImageColoring/gray/'
files = os.listdir(testpath)

files

['109.jpg', '107.jpg', '106.jpg', '110.jpg', '108.jpg']

In [11]:
# Iterate through files in the 'testpath' directory
for idx, file in enumerate(files):
    # Load and preprocess the image
    test = img_to_array(load_img(testpath + file))
    test = resize(test, (224, 224), anti_aliasing=True)
    test *= 1.0/255

    # Convert the RGB image to LAB color space
    lab = rgb2lab(test)
    l = lab[:, :, 0]

    # Create the L channel image
    L = gray2rgb(l)
    L = L.reshape((1, 224, 224, 3))

    # Generate VGG16 predictions
    vggpred = newmodel.predict(L)

    # Generate color predictions using the model
    ab = model.predict(vggpred)
    ab = ab * 128

    # Create the final LAB image
    cur = np.zeros((224, 224, 3))
    cur[:, :, 0] = l
    cur[:, :, 1:] = ab

    # Save the colorized image
    imsave('ImageColoring/MyDrive/ImageColoring/gray/' + str(idx) + ".jpg", lab2rgb(cur))


/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [ ]:
!pip freeze > ImageColoring/MyDrive/ImageColoring/requirements.txt

---